In [1]:
import re
import math
import pandas as pd
from googleapiclient.discovery import build
import configparser
import os
from pprint import pprint
# (Optional) Only needed in Colab for display()
from IPython.display import display

In [2]:
config = configparser.ConfigParser()
config.read_file(open('Test.cfg'))
print(config['My Section Header']['MyPassword'])

OurSecretPassword


In [3]:
os.environ["Our Secret Password in OS"] = config['My Section Header']['MyPassword']
print(os.environ["Our Secret Password in OS"])

OurSecretPassword


In [4]:
env_var = os.environ
pprint(dict(env_var))

{'ALLUSERSPROFILE': 'C:\\ProgramData',
 'APPDATA': 'C:\\Users\\Admin\\AppData\\Roaming',
 'APPLICATION_INSIGHTS_NO_STATSBEAT': 'true',
 'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_19036_WLMBBKTCWYNICQNI',
 'CLICOLOR': '1',
 'CLICOLOR_FORCE': '1',
 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
 'COMPUTERNAME': 'DESKTOP-OECHOOP',
 'COMSPEC': 'C:\\Windows\\system32\\cmd.exe',
 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
 'ELECTRON_RUN_AS_NODE': '1',
 'FORCE_COLOR': '1',
 'FPS_BROWSER_APP_PROFILE_STRING': 'Internet Explorer',
 'FPS_BROWSER_USER_PROFILE_STRING': 'Default',
 'GIT_PAGER': 'cat',
 'HOMEDRIVE': 'C:',
 'HOMEPATH': '\\Users\\Admin',
 'JPY_INTERRUPT_EVENT': '2700',
 'LOCALAPPDATA': 'C:\\Users\\Admin\\AppData\\Local',
 'LOGONSERVER': '\\\\DESKTOP-OECHOOP',
 'MPLBACKEND': 'module://matplotlib_inline.backend_inline',
 'NUMBE

In [5]:
# load API key into Colab
config = configparser.ConfigParser()
config.read_file(open('YoutubeAPI2.cfg'))

os.environ["Youtube_API_Access_KEY"]= config['Youtube API']['API_KEY']

In [6]:
# this is just one of the easiest ways to connect to Youtube API
youtube = build( 'youtube', 'v3', developerKey = os.environ["Youtube_API_Access_KEY"] )

In [7]:
# Clean comment text
def clean_text(s: str) -> str:
    if s is None:
        return ""
    return (s.replace("\n", " ")
             .replace("\t", " ")
             .strip())

def parse_channel_url(url: str):
    """
    Supports:
      - https://www.youtube.com/channel/UCxxxx
      - https://www.youtube.com/user/LegacyUser
      - https://www.youtube.com/@handle   (best-effort)
    """
    url = url.strip()

    m = re.search(r"youtube\.com/channel/(UC[\w-]+)", url)
    if m:
        return ("channel", m.group(1))

    m = re.search(r"youtube\.com/user/([\w-]+)", url)
    if m:
        return ("user", m.group(1))

    m = re.search(r"youtube\.com/@([\w\.\-]+)", url)
    if m:
        return ("handle", m.group(1))

    raise ValueError("Unsupported channel URL. Use /channel/UC..., /user/..., or /@handle")

def resolve_channel_id(channel_url: str) -> str:
    kind, value = parse_channel_url(channel_url)

    if kind == "channel":
        return value

    if kind == "user":
        resp = youtube.channels().list(part="id", forUsername=value).execute()
        items = resp.get("items", [])
        if not items:
            raise ValueError("Cannot resolve /user/ username to a channelId.")
        return items[0]["id"]

    if kind == "handle":
        # Best-effort: search for the channel
        resp = youtube.search().list(part="snippet", q=value, type="channel", maxResults=5).execute()
        items = resp.get("items", [])
        if not items:
            raise ValueError("Cannot resolve @handle to a channelId (no matches).")
        return items[0]["snippet"]["channelId"]

    raise ValueError("Unexpected URL type")

def get_recent_video_ids(channel_id: str, max_videos: int = 25) -> list:
    """
    Get most recent videos for a channel.
    """
    video_ids = []
    token = None

    while len(video_ids) < max_videos:
        resp = youtube.search().list(
            part="id",
            channelId=channel_id,
            type="video",
            order="date",
            maxResults=min(50, max_videos - len(video_ids)),
            pageToken=token
        ).execute()

        for it in resp.get("items", []):
            vid = it.get("id", {}).get("videoId")
            if vid:
                video_ids.append(vid)

        token = resp.get("nextPageToken")
        if not token:
            break

    return video_ids


In [8]:
## 2) Pull comments + replies for ONE video
def fetch_video_comments(video_id: str):
    # Lists for original top-level comments
    Original_author_id = []
    Original_author_name = []
    Original_comment_id = []
    Original_thread_id = []
    Original_likeCount = []
    Original_totalReplyCount = []

    # Lists for replies
    Reply_author_id = []
    Reply_author_name = []
    Reply_id = []
    Reply_parent_comment_id = []
    Reply_likeCount = []

    # Direct mapping for edges
    Reply_to_original_author_id = []

    nextPageToken = None

    while True:
        response = youtube.commentThreads().list(
            part="id,snippet",
            order="time",
            videoId=video_id,
            maxResults=100,
            pageToken=nextPageToken
        ).execute()

        for item in response.get("items", []):
            thread_id = item.get("id")

            top = item["snippet"]["topLevelComment"]
            top_snip = top["snippet"]

            top_comment_id = top.get("id")
            if not top_comment_id:
                continue

            author_obj = top_snip.get("authorChannelId")
            if author_obj is None:
                continue

            top_author_id = author_obj.get("value", None)
            if not top_author_id:
                continue

            top_author_name = top_snip.get("authorDisplayName", "Unknown")
            total_reply_count = int(item["snippet"].get("totalReplyCount", 0))

            # Store original
            Original_author_id.append(top_author_id)
            Original_author_name.append(top_author_name)
            Original_comment_id.append(top_comment_id)
            Original_thread_id.append(thread_id)
            Original_likeCount.append(int(top_snip.get("likeCount", 0)))
            Original_totalReplyCount.append(total_reply_count)

            # Pull replies
            if total_reply_count > 0:
                replies_token = None
                while True:
                    replies_response = youtube.comments().list(
                        part="snippet",
                        parentId=top_comment_id,
                        maxResults=100,
                        pageToken=replies_token
                    ).execute()

                    for r in replies_response.get("items", []):
                        r_snip = r["snippet"]

                        r_author_obj = r_snip.get("authorChannelId")
                        if not r_author_obj:
                            continue

                        r_author_id = r_author_obj.get("value", None)
                        if not r_author_id:
                            continue

                        r_author_name = r_snip.get("authorDisplayName", "Unknown")

                        Reply_author_id.append(r_author_id)
                        Reply_author_name.append(r_author_name)
                        Reply_id.append(r.get("id"))
                        Reply_parent_comment_id.append(top_comment_id)

                        Reply_to_original_author_id.append(top_author_id)  # EDGE target

                        Reply_likeCount.append(int(r_snip.get("likeCount", 0)))

                    replies_token = replies_response.get("nextPageToken")
                    if not replies_token:
                        break

        nextPageToken = response.get("nextPageToken")
        if not nextPageToken:
            break

    return {
        "Original_author_id": Original_author_id,
        "Original_author_name": Original_author_name,
        "Original_comment_id": Original_comment_id,
        "Original_thread_id": Original_thread_id,
        "Original_likeCount": Original_likeCount,
        "Original_totalReplyCount": Original_totalReplyCount,
        "Reply_author_id": Reply_author_id,
        "Reply_author_name": Reply_author_name,
        "Reply_id": Reply_id,
        "Reply_parent_comment_id": Reply_parent_comment_id,
        "Reply_likeCount": Reply_likeCount,
        "Reply_to_original_author_id": Reply_to_original_author_id
    }


In [9]:
# 3) Rank Top 5 Influencers (no topic, channel community)
#    Uses:
#      - replies received (from top-level totalReplyCount)
#      - likes received (from top-level likeCount)
#      - consistency (distinct videos participated)
#      - plus a network-ish signal: inDegree from reply edges

def compute_top5_influencers(all_video_results: list):
    # Aggregate engagement
    likes_received = {}
    replies_received = {}
    distinct_videos = {}
    indegree_from_replies = {}

    # Also collect edges for a reply in-degree measure
    # Edge: reply_author -> original_author
    for res in all_video_results:
        vid = res["video_id"]

        # originals: likes + replies received
        for u, lk, rp in zip(res["Original_author_id"], res["Original_likeCount"], res["Original_totalReplyCount"]):
            likes_received[u] = likes_received.get(u, 0) + int(lk)
            replies_received[u] = replies_received.get(u, 0) + int(rp)
            distinct_videos.setdefault(u, set()).add(vid)

        # repliers also count as participants
        for u in res["Reply_author_id"]:
            distinct_videos.setdefault(u, set()).add(vid)

        # in-degree counts from reply edges
        for src, tgt in zip(res["Reply_author_id"], res["Reply_to_original_author_id"]):
            indegree_from_replies[tgt] = indegree_from_replies.get(tgt, 0) + 1
            # also ensure src exists in dicts for later scoring
            indegree_from_replies.setdefault(src, indegree_from_replies.get(src, 0))

    # Build candidate set
    all_users = set(distinct_videos.keys()) | set(likes_received.keys()) | set(replies_received.keys()) | set(indegree_from_replies.keys())

    # Score
    rows = []
    for u in all_users:
        lk = likes_received.get(u, 0)
        rp = replies_received.get(u, 0)
        dv = len(distinct_videos.get(u, set()))
        indeg = indegree_from_replies.get(u, 0)

        # Simple, defendable scoring:
        # - likes and replies show engagement
        # - distinct videos shows consistency
        # - indegree shows "others reply to them"
        score = (
            0.40 * math.log1p(lk) +
            0.35 * math.log1p(rp) +
            0.15 * math.log1p(dv) +
            0.10 * math.log1p(indeg)
        )

        rows.append({
            "channelId": u,
            "score": score,
            "likesReceived": lk,
            "repliesReceived": rp,
            "distinctVideos": dv,
            "replyInDegree": indeg
        })

    df = pd.DataFrame(rows).sort_values("score", ascending=False).head(5).reset_index(drop=True)

    # Enrich top 5 with channel names + subs (if public)
    ids = df["channelId"].tolist()
    ch = youtube.channels().list(part="snippet,statistics", id=",".join(ids)).execute()

    id_to_name = {}
    id_to_subs = {}
    for item in ch.get("items", []):
        cid = item["id"]
        id_to_name[cid] = item["snippet"].get("title", "")
        id_to_subs[cid] = item.get("statistics", {}).get("subscriberCount", "hidden")

    df["name"] = df["channelId"].map(id_to_name).fillna("")
    df["subscriberCount"] = df["channelId"].map(id_to_subs).fillna("hidden")

    # reorder columns
    df = df[["name", "channelId", "subscriberCount", "score",
             "likesReceived", "repliesReceived", "replyInDegree", "distinctVideos"]]

    return df

In [10]:
# 4) Main: Channel URL -> Top 5 influencers

def channel_url_to_top5(channel_url: str, max_videos: int = 25):
    channel_id = resolve_channel_id(channel_url)
    video_ids = get_recent_video_ids(channel_id, max_videos=max_videos)

    print(f"Resolved channelId: {channel_id}")
    print(f"Scanning {len(video_ids)} recent videos...")

    all_results = []
    for vid in video_ids:
        res = fetch_video_comments(vid)
        res["video_id"] = vid
        all_results.append(res)

    top5 = compute_top5_influencers(all_results)
    return top5


In [ ]:
# 5) Run and test


# Paste your channel URL here:
# Examples:
# "https://www.youtube.com/channel/UCxxxx"
# "https://www.youtube.com/@somehandle"
# "https://www.youtube.com/user/legacyname"

channel_url = "https://www.youtube.com/channel/UCtxD0x6AuNNqdXO9Wp5GHew"
top5_df = channel_url_to_top5(channel_url, max_videos=25)
top5_df = top5_df.reset_index(drop=True)
top5_df.index += 1
top5_df.index.name = "Rank"
display(top5_df)

Resolved channelId: UCtxD0x6AuNNqdXO9Wp5GHew
Scanning 25 recent videos...
